In [4]:
import pandas as pd
import random
import numpy as np
import scanpy as sc
# import scipy
from tqdm import tqdm

data_path = '/data2/hratch/immune_CCI/covid/covid_atlas/'
load_h5 = True

In [ ]:
pbmc_covid = sc.read_mtx(data_path + 'raw/GSE158055_covid19_counts.mtx.gz') # raw counts
if load_h5: 
    pch5 = sc.read_h5ad(data_path + 'raw/COVID19_ALL.h5ad') # load dataset
# pbmc_covid = sc.read_10x_mtx(data_path + 'raw/counts/')

print('Finished loading covid datasets')

In [ ]:
# exclude samples with fewer than 2000 cells
md_cell = pd.read_csv(data_path + 'raw/GSE158055_cell_annotation.csv.gz')
n_samples = md_cell.sampleID.value_counts()
samples_to_keep = n_samples[n_samples > 2000].index.tolist()




In [ ]:
md = pd.read_excel(data_path + 'raw/GSE158055_sample_metadata.xlsx', sheet_name = 0, skiprows=20)
md = md.iloc[range(304 - 20), range(25)]
md = md[md['Sample name'].isin(samples_to_keep)]


contexts = md['characteristics: CoVID-19 severity'].unique()
n_contexts = contexts.shape[0]

context_counts = md['characteristics: CoVID-19 severity'].value_counts() 
min_context_type = context_counts[context_counts == context_counts.min()].index.tolist()[0]
min_context_count = len(md[md['characteristics: CoVID-19 severity'] == min_context_type]['Patients'].unique())

max_samples = min_context_count*n_contexts

context_map = {context: md[md['characteristics: CoVID-19 severity'] == \
                           context][['Sample name', 'Patients']].reset_index(drop = True) for context in contexts}

In [ ]:
# randomly select samples subsetted from the entire dataset
# make sure to choose an even number of each context
# make sure not to repeat patients within a context

n_iter = 1 # number of times to run subsetting
seed = 0

Samples = pd.DataFrame(columns = ['iteration', 'n_samples', 'sample_names'])
idx = 0

sample_iters = [3, 6, 12, 24, 36, 48, 60, 75]#list(range(n_contexts, max_samples + 1, n_contexts))

for iteration in range(n_iter):
    for n_samples in sample_iters:
        cmap_temp = context_map.copy()
        n_sample_per_context = int(n_samples/n_contexts)
        samples = list()
        for context in contexts:
            df = context_map[context].sample(frac=1, random_state = seed).drop_duplicates(subset = 'Patients') # shuffle rows to randomly drop duplicates
            samples += df.sample(n = n_sample_per_context, random_state = seed)['Sample name'].tolist()
            seed += 1
        Samples.loc[idx,:] = [iteration, n_samples, samples]
        idx += 1

Samples['sample_names'] = Samples.sample_names.apply(lambda x: '; '.join(x))            
Samples.to_csv(data_path + 'interim/timing_inputs/samples_for_timing.csv')            

In [ ]:
#cell_ids = pd.read_csv(data_path + 'raw/GSE158055_covid19_barcodes.tsv.gz', header = None)
gene_ids = pd.read_csv(data_path + 'raw/GSE158055_covid19_features.tsv.gz', header = None)
md_cell.set_index('cellName', drop = True, inplace = True)

pbmc_covid = pbmc_covid.transpose() 

In [ ]:
if load_h5:
    pbmc_covid.obs = pch5.obs
    pbmc_covid.var = pch5.var
else:
    pbmc_covid.obs = md_cell
    pbmc_covid.var = gene_ids.set_index(0, drop = True)

In [ ]:
# split by sample id 

def flatten_list(t):
    return [item for sublist in t for item in sublist]

def create_raw_counts(sample_id):
    df = pbmc_covid[pbmc_covid.obs.sampleID == sample_id]
    sc.pp.filter_cells(df, min_genes=50) 
#     sc.pp.filter_genes(df, min_cells = 3) # avoid filtering genes, will need intersection of remaining genes, which filters to many out when subsequently filtering for LR pairs
    return df

sample_ids = list(set(flatten_list([sn.split('; ') for sn in Samples.sample_names.tolist()])))
sample_counts = {sample_id: create_raw_counts(sample_id) for sample_id in sample_ids}


min_cells_to_keep = min([df.n_obs for sample_id, df in sample_counts.items()])

In [ ]:
# seed = 24
seed += 1

# subset to min cells_to_keep and write to csv
cells_to_keep = list()
for sample_id in tqdm(sample_counts):
    random.seed(seed)
    df = sample_counts[sample_id]
    df = df[df.obs.index.isin(random.sample(df.obs.index.tolist(), min_cells_to_keep))] # subset
    df.to_df().to_csv(data_path + 'interim/umi_for_timing/' + sample_id + '.csv') # write
    cells_to_keep += df.obs.index.tolist()
    seed += 1

pbmc_covid.obs[pbmc_covid.obs.index.isin(cells_to_keep)].to_csv(data_path + 'interim/timing_inputs/metadata_for_timing.csv')

# Classification

In [17]:
md = pd.read_excel(data_path + 'raw/GSE158055_sample_metadata.xlsx', sheet_name = 0, skiprows=20)
md = md.iloc[range(304 - 20), range(25)]
md = md[md['Sample name'].isin(samples_to_keep)] # still filter for datasets with > 2000 cells

In [27]:
sample_ids = md['Sample name'].unique().tolist()
pbmc_covid.obs[pbmc_covid.obs.sampleID.isin(sample_ids)].to_csv(data_path + 'interim/classification_inputs/metadata.csv')
sample_counts = {sample_id: create_raw_counts(sample_id) for sample_id in tqdm(sample_ids)}

In [37]:
for sample_id in tqdm(sample_counts):
    df = sample_counts[sample_id]
#     df.to_df().to_csv(data_path + 'interim/umi_for_classification/' + sample_id + '.csv') # write
    df.to_df().to_hdf(data_path + 'interim/classification_inputs/umi_per_sample.h5', key = sample_id)

  2%|▏         | 4/227 [00:17<18:54,  5.09s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M043-1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
  2%|▏         | 5/227 [00:25<21:38,  5.85s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M048'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
  3%|▎         | 6/227 [00:32<22:51,  6.20s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Pytho

 11%|█         | 24/227 [02:16<18:12,  5.38s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-HC012'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 11%|█         | 25/227 [02:22<18:27,  5.48s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M055'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 11%|█▏        | 26/227 [02:27<17:51,  5.33s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Pyt

 19%|█▉        | 44/227 [04:36<15:02,  4.93s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-S062'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 20%|█▉        | 45/227 [04:39<13:33,  4.47s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-S063'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 20%|██        | 46/227 [04:43<12:46,  4.23s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Pyth

 28%|██▊       | 64/227 [05:52<10:15,  3.78s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-S035-3'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*`; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 29%|██▊       | 65/227 [05:54<09:08,  3.38s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-S035-4'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 29%|██▉       | 66/227 [05:58<08:54,  3.32s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Py

 37%|███▋      | 84/227 [08:49<26:51, 11.27s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-S086-2'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 37%|███▋      | 85/227 [09:03<28:51, 12.19s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M077'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 38%|███▊      | 86/227 [09:11<25:19, 10.78s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Py

...

 63%|██████▎   | 144/227 [15:52<10:23,  7.51s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M007-4'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*`; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 64%|██████▍   | 145/227 [15:56<08:55,  6.53s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M007-5'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 64%|██████▍   | 146/227 [16:03<08:57,  6.64s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid

 72%|███████▏  | 164/227 [17:52<07:07,  6.78s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M017'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*`; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 73%|███████▎  | 165/227 [18:01<07:39,  7.42s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M019'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 73%|███████▎  | 166/227 [18:10<07:50,  7.71s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Pyt

 81%|████████  | 184/227 [21:07<08:16, 11.54s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M039-2'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 81%|████████▏ | 185/227 [21:23<08:55, 12.74s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-M040-1'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 82%|████████▏ | 186/227 [21:39<09:25, 13.79s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a val

 90%|████████▉ | 204/227 [23:41<02:21,  6.15s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-S078'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*`; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 90%|█████████ | 205/227 [23:47<02:19,  6.35s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-S079'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 91%|█████████ | 206/227 [23:53<02:08,  6.12s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Pyt

 99%|█████████▊| 224/227 [26:43<00:41, 13.69s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-S051'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
 99%|█████████▉| 225/227 [26:50<00:23, 11.75s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid Python identifier: 'S-S052'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
100%|█████████▉| 226/227 [26:58<00:10, 10.77s/it]/home/hratch/anaconda3/envs/cellchat/lib/python3.9/site-packages/tables/path.py:155: NaturalNameWarning: object name is not a valid P

In [38]:
sample_id

'S-S053'